In [ ]:
T = 2 #number of iterations for each p_obs

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
from numpy import linalg as LA
from sklearn.cluster import KMeans
import subprocess
import time
import math
import pandas as pd
import pdb

class Data_Generator:
    """Generates synthetic data"""
    def __init__(self, alpha, beta, p_obs, p_list, k, n, m):
        ## storing input parameters within the class
        self.alpha = alpha
        self.beta = beta
        self.p_obs = p_obs
        self.p_list = p_list
        self.k = k
        self.n = n
        self.m = m
        ## define additional class variables
        self.U = np.zeros((k, m)) # u_1, u_2, ..., u_K
        self.U_set = False
        self.n_per_cluster = int(n/k)
        self.cluster_id = np.arange(n)//self.n_per_cluster # 000111...(k-1)(k-1)...
        
        self.M_ground_truth = {}
        self.M_train = {}
        
        self.M_train = None
        self.Adj_list = None
        
        assert( self.n % self.k == 0 ) # Equal division possible
        
    def set_U(self, U):
        self.U = U
        self.U_set = True

    def generate_rating_data(self):
        if self.U_set:
            X_full_obs = -1+2*np.array(np.random.random((self.n,self.m)) <= np.repeat(self.U, np.array(np.ones(self.k)*self.n_per_cluster, dtype=int), axis=0), dtype=float)
            X_partial_obs = X_full_obs * np.array(np.random.random((self.n,self.m)) <= self.p_obs, dtype=float)
            return X_partial_obs
        else:
            print("U is not set yet")
            return None
    
    def generate_graph(self):
        alpha = self.alpha
        beta = self.beta
        n = self.n
        n_per_cluster = self.n_per_cluster
        cluster_id = self.cluster_id
        Adj_list = [[] for i in range(3)]

                        
        for i in range(n):
            for j in range(i+1,n):
                if cluster_id[i] == cluster_id[j]:
                    if np.random.rand() <= alpha:
                        Adj_list[0].append(float(1))
                        Adj_list[1].append(i)
                        Adj_list[2].append(j)
                        Adj_list[0].append(float(1))
                        Adj_list[1].append(j)
                        Adj_list[2].append(i)
                else:
                    if np.random.rand() <= beta:
                        Adj_list[0].append(float(1))
                        Adj_list[1].append(i)
                        Adj_list[2].append(j)
                        Adj_list[0].append(float(1))
                        Adj_list[1].append(j)
                        Adj_list[2].append(i)                        
        
        return Adj_list

In [ ]:
class CVR:
    MAX_N_OF_REFINEMENT_STEPS = 10
    
    def __init__(self, M_obs, Adj_list, n, m, k, p_gt):
        self.M_obs = M_obs
        self.M_obs_locations = np.where(M_obs != 0)
        self.Adj_list = Adj_list
        self.n = n
        self.m = m
        self.k = k #number of clusters of users
        self.d = p_gt.size
                
    def spectral_clustering_and_vote(self, truncation_threshold = 6, local_refinement_flag = False):
        M_obs = self.M_obs
        M_obs_locations = self.M_obs_locations
        Adj_list = self.Adj_list
        
        Adj = sp.coo_matrix((Adj_list[0], (Adj_list[1], Adj_list[2])))
        Adj_csr = Adj.tocsr()
        Adj_original_csr = Adj_csr
        Adj_original = np.copy(Adj)
        

        n = self.n
        m = self.m
        k = self.k
        d = self.d # number of probabilities p_1,...,p_d
        z = 2 # number of possible ratings binary in Alg 1, but will be bigger than 2 in experiment 3
        
        neighbor_list = [[] for x in range(n)]
        for i in range(len(Adj_list[0])):
            neighbor_list[Adj_list[1][i]].append(Adj_list[2][i])        
        
        # Stage 1. Spectral clustering
        # Caution: This may be slow for very large n
        deg_th = truncation_threshold * np.sum(Adj)/n
        heavy_rows = np.where(np.sum(Adj,1) > deg_th)[0]
        
        if len(heavy_rows): 
            Adj_list_new = [[] for x in range(3)]
            for i in range(len(Adj_list[0])):
                if Adj_list[1][i] in heavy_rows or Adj_list[2][i] in heavy_rows:
                    continue
                Adj_list_new[0].append(float(1))
                Adj_list_new[1].append(Adj_list[1][i])    # need to change order of i and 1 ?
                Adj_list_new[2].append(Adj_list[2][i])
        else:
            Adj_list_new = Adj_list
            
        Adj = sp.coo_matrix((Adj_list_new[0], (Adj_list_new[1], Adj_list_new[2])))
        dd, vv = sp.linalg.eigs(Adj, k = k)
        kmeans = KMeans(n_clusters=k, random_state=0).fit(np.real(vv))
        k_mean_results = kmeans.labels_        

#        print("Stage 1 results", k_mean_results)
        stage1_clustering_results = np.copy(k_mean_results)        
        
       
        
        
        # Stage 2. Majority voting
#         k_mean_results = 1-np.array(np.floor(np.arange(0,n)/(n/2)), dtype=int)
        
        B_est = np.zeros((k, m)) # Caution: The row indices of B_est and B do not match in general
        n_ct = np.zeros((k, m))
        B_ct = np.zeros((k, m, z)) # B_ct(:,:,0) for 0, B_ct(:,:,1) = for 1, and so on, used for finding p_hat
        R_ct = np.zeros((k, m, d)) 
        R_est = np.zeros((k, m)) # estimation of rating matrix from stage 2; u_hat, v_hat
        
        
        p_hat = np.zeros(d)
            
        for i in range(d):
            p_hat[i] = p_gt[i]
        
            
#         pdb.set_trace()
        
        for z in range(len(M_obs_locations[0])): # O(pnm)
            i = M_obs_locations[0][z] # i
            j = M_obs_locations[1][z] # j
            for l in range(d):
                cluster_idx = k_mean_results[i]
                if M_obs[i,j] == -1:
                    R_ct[cluster_idx, j, l] += -np.log(1-p_hat[l])    #use 1.01 instead of 1 to avoid log(0) case
                else:
                    R_ct[cluster_idx, j, l] += -np.log(p_hat[l])    #use 0.01 instead of 0 to avoid log(0) case
        
        if np.sum(p_hat < 0) or np.sum(p_hat > 1):
            pdb.set_trace()
            
        emp = []
        
        for i in range(k):
            for j in range(m):
                for l in range(d):
                    emp.append(R_ct[i, j, l])
                R_est[i, j] = p_hat[np.argmin(emp)]
                emp = []
                                
#        print("Stage 2 results", R_est)


        # Stage 3. Local refinement
        observed_entries = [None for i in range(n)]
        row_sums = Adj_csr.sum(axis=1)        
#        print("row_sums", row_sums)

#         for i in range(n):
#             observed_entries[i] = np.where(~np.isnan(M_train_arr[i,:]))
        
        stage3_clustering_results = np.copy(k_mean_results)
        edges_per_cluster = np.zeros((n, k))
        weighted_sum_of_correct_ratings_per_cluster = np.zeros((n, k))
        weighted_sum_of_incorrect_ratings_per_cluster = np.zeros((n, k))
        number_of_edges_same_cluster = 0
        number_of_edges_diff_cluster = 0
        number_of_total_pairs_same_cluster = 0
        number_of_total_pairs_diff_cluster = 0

        n_per_cluster_stage1_list = []
        for i in range(k):
            n_per_cluster_stage1_list.append(len(np.where(k_mean_results==i)[0]))
            
#        print("stage3_n", n)
#        print("stage1_n_per_cluster",n_per_cluster_stage1_list)        
        
        for i in range(k):
            number_of_total_pairs_same_cluster += n_per_cluster_stage1_list[i]*(n_per_cluster_stage1_list[i]-1)/2
            
        for i in range(k):
            for j in range(i+1,k):
                number_of_total_pairs_diff_cluster += n_per_cluster_stage1_list[i]*n_per_cluster_stage1_list[j]
        
                    

        for i in range(n):
            for each_neighbor in neighbor_list[i]:
                if k_mean_results[i] == k_mean_results[each_neighbor]:
                    number_of_edges_same_cluster += 1
                else:
                    number_of_edges_diff_cluster += 1
                    
        number_of_edges_same_cluster = number_of_edges_same_cluster/2
        number_of_edges_diff_cluster = number_of_edges_diff_cluster/2



#        print("number_of_total_pairs_same_cluster", number_of_total_pairs_same_cluster)
#        print("number_of_total_pairs_diff_cluster", number_of_total_pairs_diff_cluster)
#        print("number_of_edges_same_cluster", number_of_edges_same_cluster)
#        print("number_of_edges_diff_cluster", number_of_edges_diff_cluster)
        alpha_hat = number_of_edges_same_cluster/number_of_total_pairs_same_cluster
        beta_hat = number_of_edges_diff_cluster/number_of_total_pairs_diff_cluster

#        print("a hat", alpha_hat)
#       print("b hat", beta_hat)

    
        if local_refinement_flag:
            n_of_refinement_steps = 0

            while n_of_refinement_steps <= CVR.MAX_N_OF_REFINEMENT_STEPS:
                change_flag = False
                n_of_refinement_steps += 1
#                print(n_of_refinement_steps)
                new_k_mean_results = np.copy(stage3_clustering_results)
  
                nodes_in_each_cluster = {}
                for i in range(k):
                    nodes_in_each_cluster[i] = np.where(stage3_clustering_results == i)
#                 print nodes_in_each_cluster
                    
                if n_of_refinement_steps == 1: # initial update
                    for i in range(n):
                        for each_neighbor in neighbor_list[i]:
                            edges_per_cluster[i, stage3_clustering_results[each_neighbor]] += 1
                    list_of_changes = []                    

                    
                    for z in range(len(M_obs_locations[0])): # O(pnm)
                        i = M_obs_locations[0][z] # i
                        j = M_obs_locations[1][z] # j
                        for l in range(k):
                            if M_obs[i,j] == -1:
                                weighted_sum_of_incorrect_ratings_per_cluster[i, l] += np.log(1-R_est[l, j])     
                            else:
                                weighted_sum_of_correct_ratings_per_cluster[i, l] += np.log(R_est[l, j])    
                    
                else:
                    for i in range(n):
                        for each_change in list_of_changes: # O(n)
                            j, cluster_old, cluster_new = each_change
                            if Adj_original_csr[i,j]:
                                edges_per_cluster[i, cluster_old] -= 1
                                edges_per_cluster[i, cluster_new] += 1
#                     pdb.set_trace()
                    list_of_changes = []
#                 pdb.set_trace()

                n_per_cluster_middle_of_stage3_list = []
                for i in range(k):
                    n_per_cluster_middle_of_stage3_list.append(len(np.where(new_k_mean_results==i)[0]))

#               print("n_per_cluster_middle_of_stage3",n_per_cluster_middle_of_stage3_list) 

                for i in range(n):
#                     print(i)
                    likelihood_array = np.zeros(k)
                    
#                     edges_per_cluster = np.zeros(k)
#                     for j in range(n): # O(n^2)
#                         if Adj_original[i,j] == 1:
#                             edges_per_cluster[stage3_clustering_results[j]] += 1



                    for j in range(k): # O(n)
                        cluster_idx = j
                        deg_internal_1 = edges_per_cluster[i, j]
                        deg_internal_0 = n_per_cluster_middle_of_stage3_list[j] -1 - deg_internal_1
                        deg_external_1 = np.int(row_sums[i]) - deg_internal_1
                        deg_external_0 = n-n_per_cluster_middle_of_stage3_list[j] - deg_external_1
                        
                        
#                         print("Node %d, Cluster %d" % (i,j))
#                         print(deg_internal_1, deg_internal_0, deg_external_1, deg_external_0, weighted_sum_of_correct_ratings, weighted_sum_of_incorrect_ratings)

                        likelihood_array[j] = \
                                    np.log(alpha_hat) * deg_internal_1 + \
                                    np.log(1-alpha_hat) * deg_internal_0 + \
                                    np.log(beta_hat) * deg_external_1 + \
                                    np.log(1-beta_hat) * deg_external_0 + \
                                    weighted_sum_of_correct_ratings_per_cluster[i, j] + \
                                    weighted_sum_of_incorrect_ratings_per_cluster[i, j]
#                     pdb.set_trace()
                    opt_clustering_assignment = np.argmax(likelihood_array)
                    if opt_clustering_assignment != stage3_clustering_results[i]:
                        list_of_changes.append((i, stage3_clustering_results[i], opt_clustering_assignment))
                        new_k_mean_results[i] = opt_clustering_assignment
                        change_flag = True
                        
#                         pdb.set_trace()
#                         print "Node %d is removed from %d to %d" % (i, k_mean_results[i], opt_clustering_assignment)

                if not change_flag: # nothing happened
                    break

                stage3_clustering_results = np.copy(new_k_mean_results)                    
                

        n_per_cluster_stage3_list = []
        for i in range(k):
            n_per_cluster_stage3_list.append(len(np.where(stage3_clustering_results==i)[0]))    
            
#        print("stage3_n_per_cluster", n_per_cluster_stage3_list)
            
        return B_est, p_hat, stage1_clustering_results, stage3_clustering_results, R_est
    
    

In [ ]:
n = 10000
m = 5000
k = 2
p_gt = np.array([0.2, 0.5, 0.7])
d = p_gt.size

M_max = 0
for i in range(p_gt.size-1):
    p_1, p_2 = p_gt[i], p_gt[i+1]
    if np.sqrt(p_1 * p_2) + np.sqrt((1-p_1)*(1-p_2)) > M_max:
        M_max = np.sqrt(p_1 * p_2) + np.sqrt((1-p_1)*(1-p_2))
gamma = 0.5



U = np.array([[0.2]*int(m/4)+[0.5]*int(m/2)+[0.7]*int(m/4),[0.2]*int(m/4)+[0.5]*int(m/4)+[0.7]*int(m/4)+[0.5]*int(m/4)])

obs_rate = []
I_s_list = []
prob_err_list = []
prob_suc_list = []
p_opt_list = []

n_of_t = 10
n_of_p = 10

In [ ]:
from datetime import datetime


for i in np.arange(n_of_t):
    t = 0.5 + i*(2.0-0.5)/n_of_t # (2-0.5)/50.0
    alpha = np.power((np.sqrt(t) + 1), 2)*np.log(n)/n
    beta = np.log(n)/n
    I_s = -2*np.log(np.sqrt(alpha*beta) + np.sqrt((1-alpha)*(1-beta)))
    p_opt = max((np.log(n) - n*I_s/2)/(gamma*m), 2*np.log(m)/n)/(1-M_max)
    for j in range(n_of_p):
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(j)
        print("Current Time =", current_time)
        p_obs = 0.05 + j*(0.15-0.05)/n_of_p 
        prob_ct = 0
        for h in range(T):
            dg = Data_Generator(alpha, beta, p_obs, p_gt, k, n, m)
            dg.set_U(U)
            Adj_list = dg.generate_graph()
            solver = CVR(dg.generate_rating_data(), Adj_list, n, m, k, p_gt)
            B_est, p_hat, stage1_clustering_results, stage3_clustering_results, R_est = solver.spectral_clustering_and_vote(local_refinement_flag = True)
            max_err = 0
            ground_truth_cluster = np.array(np.floor(np.arange(0,n)/(n/2)), dtype=int)
            for i in range(n):
                if np.max(np.abs(R_est[stage3_clustering_results[i]] - U[ground_truth_cluster[i]])) > max_err:
                    max_err = np.max(np.abs(R_est[stage3_clustering_results[i]] - U[ground_truth_cluster[i]]))
            
            if max_err != 0:
                prob_ct += 1
                
        prob_err = prob_ct/T
        prob_suc = 1 - prob_err
        
        obs_rate.append(p_obs)
        I_s_list.append(I_s)
        prob_err_list.append(prob_err)
        prob_suc_list.append(prob_suc)
        p_opt_list.append(p_opt)
        
print("obs_rate", obs_rate)
print("I_s_list", I_s_list)
print("prob_err_list", prob_err_list)
print("prob_suc_list", prob_suc_list)
print("p_opt_list", p_opt_list)

In [ ]:
import scipy.io as io

io.savemat('Figure_2_a_data',{'obs':obs_rate, 'I_s':I_s_list, 'prob_err':prob_err_list, 'prob_suc':prob_suc_list, 'p_opt':p_opt_list})

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.sparse as sp
from numpy import linalg as LA
from sklearn.cluster import KMeans
import subprocess
import time
import math
import pandas as pd
import pdb

In [ ]:
import scipy.io as io
import pandas as pd


loaded = io.loadmat('Figure_2_a_data')

I_s_min = 0.0003920299707424321
I_s_max = 0.0017784049575601882
p_opt_min = 0.13748550444397495
p_opt_max = 0.0807558504441226

I_s_extended_0 = np.append(np.array([I_s_min]), loaded['I_s'][0])
I_s_extended = np.append(I_s_extended_0, np.array([I_s_max]))
#print(I_s_extended)
#print(type(I_s_extended))

p_opt_extended_0 = np.append(np.array([p_opt_min]), loaded['p_opt'][0])
p_opt_extended = np.append(p_opt_extended_0, np.array([p_opt_max]))
#print(p_opt_extended)
#print(type(p_opt_extended))

In [ ]:
dset = pd.DataFrame({'x':loaded['I_s'][0], 'y':loaded['obs'][0], 'z':loaded['prob_err'][0], 'w':loaded['p_opt'][0]})

plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13)
plt.rc('axes', labelsize=13)
plt.rc('axes', linewidth=0.5)
mpl.rcParams['patch.linewidth']=0.5


#width = 5.98
#height = 4.92

width = 3.27
height = width / 1.29

fig, ax = plt.subplots()
fig.subplots_adjust(left= 0.21, bottom=0.2, right=0.89, top=.93)
fig.set_size_inches(width, height)
#gray_r  cividis_r
sc = plt.scatter(dset['x'], dset['y'], c=dset['z'], cmap='cividis_r', marker = 's', s = 215)
cbar = plt.colorbar(sc)
cbar.set_label(r'$\Pr(\psi_1 (N^{\Omega}, G)\neq R)$')
plt.clim(0.0,1)
plt.plot(I_s_extended, p_opt_extended, linewidth=1.5, color = 'red')
plt.xlabel(r'$I_s$')
plt.ylabel(r'$p$')
plt.xlim(np.min(dset['x'])-(np.max(dset['x'])-np.min(dset['x']))/18, np.max(dset['x'])+(np.max(dset['x'])-np.min(dset['x']))/18)
plt.ylim(np.min(dset['y'])-(np.max(dset['y'])-np.min(dset['y']))/18, np.max(dset['y'])+(np.max(dset['y'])-np.min(dset['y']))/18)

from matplotlib.ticker import FixedLocator, FixedFormatter

y_formatter = FixedFormatter(["0.06", "0.08", "0.10", "0.12", "0.14"])
y_locator = FixedLocator([0.06, 0.08, 0.10, 0.12, 0.14])
ax.yaxis.set_major_formatter(y_formatter)
ax.yaxis.set_major_locator(y_locator)

plt.locator_params(axis='x', nbins=5)


plt.savefig('Figure_2_a.eps', format='eps', dpi=1000)
plt.show()